In [2]:
from math import *

import numpy as np
import pandas as pd

import scipy

from math import *
import numpy as np
import scipy.stats

from copy import *

from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

In [3]:
data = pd.read_csv('data_prob_90.csv', header=0, delim_whitespace=False)
data_name = data.columns[1:]
data = np.array(data)

In [4]:
data_word = pd.DataFrame(data[:, 0]) # 데이터 인풋. 인풋된 데이터는 data. input 되는 것은 논문 더미들.

data = np.delete(data, 0, 1) # 0번째 column은 data_word. 뽑아내고 나머지 데이타 보존.

data = np.nan_to_num(data, copy=True, nan=99) # na인 cell은 값을 99로 설정.

In [5]:
data = np.array(data, dtype=float)

In [6]:
data_m = data # 데이터 타입 매트릭스로 바뀐 데이터. 파이썬에선 pd.dataframe 사용

logit_x = np.log(data_m / (1 - data_m)) # 매트릭스 데이터를 logit化

In [7]:
from sklearn.preprocessing import MinMaxScaler

data_m = MinMaxScaler().fit_transform(data_m)

In [8]:



# <editor-fold desc="covid_20 parameters">


ndim     = 2 
niter    = 55000
nburn    = 5000
nthin    = 5
nprint   = 5000
"""

ndim = 2 #차원?
niter = 200
nburn = 0
nthin = 5 #thining lapse
nprint = 100
"""


"""
모든 topic의 분포는 theta ~ 디리클레 알파를 따름

biterm 총체 B에서 biterm b를 뽑으면, 이 biterm이 어느 topic z에 속할지는 z ~ 다항분포 세타

topic z의 topic-word 분포는 phi_z ~ 디리클레 베타를 따름
topic = z, word = w. z가 정해졌을 때 이로부터 각 단어가 가지는 확률을 모아서 set으로 한것이 phi_z.

골라진 topic에 대응하는 topic-word 분포로부터 단어 2개가 골라질 확률은 w_i, w_j ~ 다항(phi_z)를 따름
"""

jump_beta = 0.3
jump_theta = 1.0
jump_w = 0.06
jump_z = 0.50
jump_gamma = 0.01

pr_mean_beta = 0
pr_sd_beta = 1
pr_mean_theta = 0
pr_sd_theta = 1
pr_mean_gamma = 0.0
pr_sd_gamma = 1.0
pr_a_sigma = 0.001
pr_b_sigma = 0.001
pr_a_th_sigma = 0.001
pr_b_th_sigma = 0.001
# </editor-fold> #TODO 뭐지? 패러미터 스태티스틱?
#TODO 뭐지? 패러미터 스태티스틱?

----

In [9]:
data = logit_x

In [10]:
#from pyarma import *

In [11]:
missing = 99

In [12]:
def onepl_lsrm_cont_missing(data,

                            ndim,
                            niter,
                            nburn,
                            nthin,
                            nprint,

                            jump_beta,
                            jump_theta,
                            jump_gamma,
                            jump_z,
                            jump_w,

                            pr_mean_beta,
                            pr_sd_beta,
                            pr_a_th_sigma,
                            pr_b_th_sigma,
                            pr_mean_theta,

                            pr_a_sigma,
                            pr_b_sigma,
                            pr_mean_gamma,
                            pr_sd_gamma,

                            missing):
    ##############################################################################

    def update_function(beta, theta, gamma):
        return (- pow((data[k, i] - beta[i] - theta[k] + gamma * dist[k, i]), 2) /
                (2 * pow(pr_sd, 2))
                )

    def rejection_algorithm(ratio):
        if ratio > 0.0:
            accept = 1
        else:
            un = np.random.uniform(1)
            if np.log(un) < ratio:
                accept = 1
            else:
                accept = 0

        return accept

    ##############################################################################

    nsample = data.shape[0]
    nitem = data.shape[1]

    pr_mean_z = 0.0
    pr_mean_w = 0.0

    pr_sd_z = 1.0
    pr_sd_w = 1.0
    pr_sd = 1.0
    pr_sd_theta = 1.0

    oldbeta = np.random.uniform(size=nitem)
    oldtheta = np.random.uniform(size=nsample)
    oldz = np.random.uniform(size=(nsample, ndim))
    oldw = np.random.uniform(size=(nitem, ndim))

    # oldbeta = np.random.uniform(nitem)
    # oldtheta = np.random.uniform(nsample)
    # oldz = np.random.rand(nsample, ndim)
    # oldw = np.random.rand(nitem, ndim)

    oldbeta = oldbeta * 4.0 - 2.0
    oldtheta = oldtheta * 4.0 - 2.0
    oldz = oldz * 2.0 - 1.0
    oldw = oldw * 2.0 - 1.0

    newbeta = deepcopy(oldbeta)
    newtheta = deepcopy(oldtheta)
    newz = deepcopy(oldz)
    neww = deepcopy(oldw)

    ##############################################################################

    oldgamma = newgamma = 1  # gamma1 = log(gamma)

    samp_beta = np.zeros(shape=((niter - nburn) // nthin, nitem))
    samp_theta = np.zeros(shape=((niter - nburn) // nthin, nsample))

    samp_z = np.zeros(shape=((niter - nburn) // nthin, nsample, ndim))
    samp_w = np.zeros(shape=((niter - nburn) // nthin, nitem, ndim))

    samp_sd_theta = np.zeros(shape=(niter - nburn) // nthin, )
    samp_sd = np.zeros(shape=(niter - nburn) // nthin, )
    samp_mle = np.zeros(shape=(niter - nburn) // nthin, )
    samp_gamma = np.zeros(shape=(niter - nburn) // nthin, )

    accept_beta = np.zeros(shape=nitem, )
    accept_theta = np.zeros(shape=nsample, )
    accept_z = np.zeros(shape=nsample, )
    accept_w = np.zeros(shape=nitem, )

    accept_gamma = 0
    accept = 0
    count = 0

    dist = np.zeros(shape=(nsample, nitem), )

    old_dist_k = np.zeros(nitem, )
    new_dist_k = np.zeros(nitem, )
    old_dist_i = np.zeros(nsample, )
    new_dist_i = np.zeros(nsample, )

    ##############################################################################


    for iter in range(niter):
        # dist(j,i) is distance of z_j and w_i

        dist = np.where(True, 0, dist)  # 매 이터레이션마다 거리 매트릭스를 0으로 리셋

        for i in range(nitem):
            for k in range(nsample):
                dist_temp = 0.0
                for j in range(ndim):
                    dist_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
                    dist[k, i] = sqrt(dist_temp)

        # beta update
        for i in range(nitem):
            # TODO 컬럼부터 갱신하는 이유가 있는건가?

            newbeta[i] = np.random.normal(oldbeta[i], jump_beta, 1)
            old_like_beta = new_like_beta = 0.0

            for k in range(nsample):
                if data[k, i] != missing:
                    new_like_beta += update_function(newbeta, oldtheta, oldgamma)
                    old_like_beta += update_function(oldbeta, oldtheta, oldgamma)
                    

            num = (new_like_beta +
                   scipy.stats.norm.logpdf(newbeta[i], pr_mean_beta, pr_sd_beta))
            den = (old_like_beta +
                   scipy.stats.norm.logpdf(oldbeta[i], pr_mean_beta, pr_sd_beta))
            ratio = num - den

            accept = rejection_algorithm(ratio)


            if accept == 1:
                oldbeta[i] = newbeta[i]
                accept_beta[i] += 1.0 / (niter * 1.0)

            else:
                newbeta[i] = oldbeta[i]

        # theta update
        for k in range(nsample):
            newtheta[k] = np.random.normal(oldtheta[k], jump_theta, 1)
            old_like_theta = new_like_theta = 0.0

            for i in range(nitem):
                if data[k, i] != missing:
                    new_like_theta += update_function(oldbeta, newtheta, oldgamma)
                    old_like_theta += update_function(oldbeta, oldtheta, oldgamma)

            num = (new_like_theta +
                   scipy.stats.norm.logpdf(newtheta[k], pr_mean_theta, pr_sd_theta))
            den = (old_like_theta +
                   scipy.stats.norm.logpdf(oldtheta[k], pr_mean_theta, pr_sd_theta))
            ratio = num - den

            accept = rejection_algorithm(ratio)

            if accept == 1:
                oldtheta[k] = newtheta[k]
                accept_theta[k] += 1.0 / (niter * 1.0)
            else:
                newtheta[k] = oldtheta[k]

        # gamma(log(gamma)) update

        newgamma = np.random.lognormal(log(oldgamma), jump_gamma, 1)
        old_like_gamma = new_like_gamma = 0.0

        for k in range(nsample):
            for i in range(nitem):
                if data[k, i] != missing:
                    new_like_gamma += update_function(oldbeta, newtheta, newgamma)
                    old_like_gamma += update_function(oldbeta, newtheta, oldgamma)

        num = (new_like_gamma +
               scipy.stats.lognorm.logpdf(oldgamma, s=jump_gamma, loc=0, scale=exp(log(newgamma))) +
               scipy.stats.lognorm.logpdf(newgamma, s=pr_sd_gamma, loc=0, scale=exp(pr_mean_gamma))
               )
        den = (old_like_gamma +
               scipy.stats.lognorm.logpdf(newgamma, s=jump_gamma, loc=0, scale=exp(log(oldgamma))) +
               scipy.stats.lognorm.logpdf(oldgamma, s=pr_sd_gamma, loc=0, scale=exp(pr_mean_gamma))
               )
        ratio = num - den

        accept = rejection_algorithm(ratio)

        if accept == 1:
            oldgamma = newgamma
            accept_gamma += 1.0 / (niter * 1.0)
        else:
            newgamma = oldgamma

        # zj update

        for k in range(nsample):
            for j in range(ndim):
                newz[k, j] = np.random.normal(oldz[k, j], jump_z, 1)
            old_like_z = new_like_z = 0.0

            # calculate distance of oldw and newz

            for i in range(nitem):
                dist_old_temp = dist_new_temp = 0.0
                for j in range(ndim):
                    dist_new_temp += pow((newz[k, j] - oldw[i, j]), 2.0)
                    dist_old_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
                new_dist_k[i] = sqrt(dist_new_temp)
                old_dist_k[i] = sqrt(dist_old_temp)

            # calculate likelihood

            for i in range(nitem):
                if data[k, i] != missing:
                    new_like_z += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * new_dist_k[i]), 2) /
                                   (2 * pow(pr_sd, 2))
                                   )
                    old_like_z += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * old_dist_k[i]), 2) /
                                   (2 * pow(pr_sd, 2))
                                   )

            num = den = 0.0

            for j in range(ndim):
                num += scipy.stats.norm.logpdf(newz[k, j], pr_mean_z, pr_sd_z)
                den += scipy.stats.norm.logpdf(oldz[k, j], pr_mean_z, pr_sd_z)

            # Rprintf("%.3f %.3f %.3f %.3f\n", num, den, new_like_z, old_like_z)
            # arma::dvec newzz = dmvnorm(newz.cols(2*j,2*j+1),pr_mean_z,pr_cov_z,TRUE)
            # arma::dvec oldzz = dmvnorm(oldz.cols(2*j,2*j+1),pr_mean_z,pr_cov_z,TRUE)

            num += new_like_z
            den += old_like_z
            ratio = num - den

            accept = rejection_algorithm(ratio)

            if accept == 1:
                for j in range(ndim):
                    oldz[k, j] = newz[k, j]
                accept_z[k] += 1.0 / (niter * 1.0)

            else:
                for j in range(ndim):
                    newz[k, j] = oldz[k, j]

                #  wi update
        for i in range(nitem):
            for j in range(ndim):
                neww[i, j] = np.random.normal(oldw[i, j], jump_w, 1)
            old_like_w = new_like_w = 0.0

            # calculate distance of neww and oldz

            for k in range(nsample):
                dist_old_temp = dist_new_temp = 0.0
                for j in range(ndim):
                    dist_new_temp += pow((oldz[k, j] - neww[i, j]), 2.0)  # TODO: Why Old - New?
                    dist_old_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
                new_dist_i[k] = sqrt(dist_new_temp)
                old_dist_i[k] = sqrt(dist_old_temp)

            # calculate likelihood

            for k in range(nsample):
                if data[k, i] != missing:
                    new_like_w += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * new_dist_i[k]), 2) /
                                   (2 * pow(pr_sd, 2))
                                   )
                    old_like_w += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * old_dist_i[k]), 2) /
                                   (2 * pow(pr_sd, 2))
                                   )

            num = den = 0.0

            for j in range(ndim):
                num += scipy.stats.norm.logpdf(neww[i, j], pr_mean_w, pr_sd_w)
                den += scipy.stats.norm.logpdf(oldw[i, j], pr_mean_w, pr_sd_w)

            num += new_like_w
            den += old_like_w
            ratio = num - den

            accept = rejection_algorithm(ratio)

            if accept == 1:
                for j in range(ndim):
                    oldw[i, j] = neww[i, j]
                accept_w[i] += 1.0 / (niter * 1.0)
            else:
                for j in range(ndim):
                    neww[i, j] = oldw[i, j]

        # sigma_theta update with gibbs

        post_a_th_sigma = pr_a_th_sigma * 2 + nsample
        post_b_th_sigma = pr_b_th_sigma

        for j in range(nsample):  # TODO: 여기 j인데 nsample인거 맞음?
            post_b_th_sigma += pow((oldtheta[j] - pr_mean_theta), 2.0)
        pr_sd_theta = sqrt(2 * post_b_th_sigma * (1.0 / np.random.chisquare(post_a_th_sigma)))

        # dist(j,i) is distance of z_j and w_i

        dist = np.where(True, 0, dist)

        for i in range(nitem):
            for k in range(nsample):
                dist_temp = 0.0
                for j in range(ndim):
                    dist_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
                dist[k, i] = sqrt(dist_temp)

        # sigma update with gibbs

        post_a_sigma = pr_a_sigma * 2 + nsample * nitem
        post_b_sigma = pr_b_sigma

        for j in range(nsample):  # TODO: 여기 j인데 nsample인거 맞음?
            for i in range(nitem):
                post_b_sigma += pow((data[j, i] - oldbeta[i] - oldtheta[j] + oldgamma * dist[j, i]), 2.0) / 2

        pr_sd = sqrt(2 * post_b_sigma * (1.0 / np.random.chisquare(post_a_sigma)))

        # burn, thin

        if iter >= nburn and iter % nthin == 0:
            for i in range(nitem):
                samp_beta[count, i] = oldbeta[i]
            for k in range(nsample):
                samp_theta[count, k] = oldtheta[k]
            for i in range(nitem):
                for j in range(ndim):
                    samp_w[count, i, j] = oldw[i, j]
            for k in range(nsample):
                for j in range(ndim):
                    samp_z[count, k, j] = oldz[k, j]

            samp_gamma[count] = oldgamma
            samp_sd_theta[count] = pr_sd_theta
            samp_sd[count] = pr_sd

            mle = 0.0

            for i in range(nitem):
                mle += scipy.stats.norm.logpdf(oldbeta[i], pr_mean_beta, pr_sd_beta)
            for k in range(nsample):
                mle += scipy.stats.norm.logpdf(oldtheta[k], pr_mean_theta, pr_sd_theta)
            for i in range(nitem):
                for j in range(ndim):
                    mle += scipy.stats.norm.logpdf(oldw[i, j], pr_mean_w, pr_sd_w)
            for k in range(nsample):
                for j in range(ndim):
                    mle += scipy.stats.norm.logpdf(oldz[k, j], pr_mean_z, pr_sd_z)
            for k in range(nsample):
                for i in range(nitem):
                    mle += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * dist[k, i]), 2)
                            / (2 * pow(pr_sd, 2)))

            mle += scipy.stats.lognorm.logpdf(oldgamma, s=pr_sd_gamma, loc=0, scale=exp(pr_mean_gamma))
            samp_mle[count] = mle

            count += 1

        if iter % nprint == 0:
            print("Iteration: ", iter)
            print("count", count)
            for i in range(nitem):
                print("nitem: ", i, " with ", oldbeta[i])
            print("oldgamma: ", oldgamma, "     pr_sd_theta: ", pr_sd_theta)

    return {"beta": samp_beta,
            "theta": samp_theta,
            "z": samp_z,
            "w": samp_w,
            "gamma": samp_gamma,
            "sigma_theta": samp_sd_theta,
            "sigma": samp_sd,
            "map": samp_mle,
            "accept_beta": accept_beta,
            "accept_theta": accept_theta,
            "accept_z": accept_z,
            "accept_w": accept_w,
            "accept_gamma": accept_gamma}








In [ ]:
# Set 99 as missing
"""
output = onepl_lsrm_cont_missing(data,
                                 ndim, niter, nburn, nthin, nprint,
                                 jump_beta, jump_theta, jump_gamma, jump_z, jump_w,
                                 pr_mean_beta, pr_sd_beta, pr_a_th_sigma, pr_b_th_sigma, pr_mean_theta,
                                 pr_a_sigma, pr_b_sigma, pr_mean_gamma, pr_sd_gamma,
                                 99)
"""

output = onepl_lsrm_cont_missing(logit_x,
                                 ndim, niter, nburn, nthin, nprint,
                                 jump_beta, jump_theta, jump_gamma, jump_z, jump_w,
                                 pr_mean_beta, pr_sd_beta, pr_a_th_sigma, pr_b_th_sigma, pr_mean_theta,
                                 pr_a_sigma, pr_b_sigma, pr_mean_gamma, pr_sd_gamma,
                                 99)

Iteration:  0
count 0
nitem:  0  with  -1.9350401497374379
nitem:  1  with  1.5926341500302954
nitem:  2  with  1.0876471041621665
nitem:  3  with  -1.3360421796903652
nitem:  4  with  -1.8160679511004845
nitem:  5  with  -0.618587035011775
nitem:  6  with  -1.2030359507377462
nitem:  7  with  1.6567079581086346
nitem:  8  with  -1.7436899532005499
nitem:  9  with  -0.7468769587051729
nitem:  10  with  0.811541856307731
nitem:  11  with  0.1285121348927668
nitem:  12  with  -1.0871706981520526
nitem:  13  with  0.9603859975603259
nitem:  14  with  -1.7116348984500118
nitem:  15  with  1.0270767107772931
nitem:  16  with  -1.2699154586126271
nitem:  17  with  1.0489221635421644
nitem:  18  with  0.5596435838347165
nitem:  19  with  1.4566440338816675
oldgamma:  [1.01191415]      pr_sd_theta:  2.000410829259988


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Song1\anaconda3\envs\covid\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Song1\AppData\Local\Temp/ipykernel_16976/4274860929.py", line 11, in <module>
    output = onepl_lsrm_cont_missing(logit_x,
  File "C:\Users\Song1\AppData\Local\Temp/ipykernel_16976/2650868760.py", line 211, in onepl_lsrm_cont_missing
    newz[k, j] = np.random.normal(oldz[k, j], jump_z, 1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Song1\anaconda3\envs\covid\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Song1\anaconda3\envs\covid\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Song1\AppData\Local\Temp/ipykernel_16976/4274860929.py", line 11, in <module>
    output = onepl_lsrm_cont_missing(logit_x,
  File "C:\Users\Song1\AppData\Local\Temp/ipykernel_16976/2650868760.py", line 211, in onepl_lsrm_cont_missing
    newz[k, j] = np.random.normal(oldz[k, j], jump_z, 1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Song1\anaconda3\envs\covid\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\

In [16]:
output.keys()

dict_keys(['beta', 'theta', 'z', 'w', 'gamma', 'sigma_theta', 'sigma', 'map', 'accept_beta', 'accept_theta', 'accept_z', 'accept_w', 'accept_gamma'])

In [75]:
def procrustes_mine(X, X_star):
    n = X.shape[0]
    m = X.shape[1]

    J = np.identity(n)

    C = X_star.transpose() @ J @ X
    svd_out = np.linalg.svd(C)
    R = svd_out[2] @ svd_out[0].transpose()
    #C = udv', v@u'

    s = 1
    X_new = s * X @ R + np.zeros((n, m))

    return X_new

# <editor-fold desc="MCMC Process">

In [132]:
nsample = data.shape[0]
nitem  = data.shape[1]

nmcmc = int((niter - nburn) / nthin)

max_address = output['map'].argmax()

w_star = output['w'][max_address, :, :]
z_star = output['z'][max_address, :, :]

w_proc = np.zeros((nmcmc, nitem, ndim), )
z_proc = np.zeros((nmcmc, nsample, ndim), )

for iter in range(nmcmc):
    z_iter = output['z'][iter, :, :]

    if iter != max_address:
        z_proc[iter, :, :] = procrustes_mine(z_iter, z_star) #TODO ======= 210717 =======
    else: 
        z_proc[iter,:,:] = z_iter
        
        

    w_iter = output['w'][iter, :, :]

    if iter != max_address:
        w_proc[iter,:,:] = procrustes_mine(w_iter, w_star) #TODO ======= 210717 =======
    else: 
        w_proc[iter,:,:] = w_iter



w_est = np.empty((nitem, ndim))
w_est[:] = np.nan

for i in range(nitem):
    for j in range(ndim):
        w_est[i, j] = w_proc[:, i, j].mean()


z_est = np.empty((nsample, ndim),)
z_est[:] = np.nan

for k in range(nsample):
    for j in range(ndim):
        z_est[k, j] = z_proc[:, k, j].mean()


beta_est = output["beta"].mean(axis = 0)
theta_est = output["theta"].mean(axis = 0)

sigma_theta_est = output["sigma_theta"].mean()
gamma_est = output["gamma"].mean()

output_new = {"beta_estimate" : beta_est,
              "theta_estimate" : theta_est,
              "sigma_theta_estimate" : sigma_theta_est,
              "gamma_estimate" : gamma_est,
              "z_estimate" : z_est,
              "w_estimate" : w_est,
              "beta" : output["beta"],
              "theta" : output["theta"],
              "theta_sd" : output["sigma_theta"],
              "gamma" : output["gamma"],
              "z" : z_proc,
              "w" : w_proc,
              "accept_beta" : output["accept_beta"],
              "accept_theta" : output["accept_theta"],
              "accept_w" : output["accept_w"],
              "accept_z" : output["accept_z"],
              "accept_gamma" : output["accept_gamma"]
              }

-1.8622287864472156


In [ ]:
## lsrm plot
#### ggplot
"""
a = pd.DataFrame(output_new["z_estimate"], columns=["coordinate_1", "coordinate_2"])
b = pd.DataFrame(output_new["w_estimate"], columns=["coordinate_1", "coordinate_2"]) #x와 w의 coordinate로 df 구성

b["topic_name"] = data_name
b["id"] = range(1, data_m.shape[1]+1)
"""

a = pd.DataFrame(output_new["z_estimate"], columns=["coordinate_1", "coordinate_2"])
b = pd.DataFrame(output_new["w_estimate"], columns=["coordinate_1", "coordinate_2"]) #x와 w의 coordinate로 df 구성

b["topic_name"] = data_name
b["id"] = range(1, data_m.shape[1]+1)

############### Rotate


angle = -pi/30

#M = pd.DataFrame( [[cos(angle), sin(angle)], [-sin(angle), cos(angle)]], columns=["coordinate_1", "coordinate_2"])
#print(M)

M = np.array([[cos(angle), sin(angle)], [-sin(angle), cos(angle)]])

# clockwise 회전용도. 따라서 일반적인 회전행렬의 inv. clockwise 기준 -6도씩 돌림.


bnew = pd.DataFrame(np.array(-b.iloc[:, 0:2]) @ M, columns=["coordinate_1", "coordinate_2"])
bnew["topic_name"] = b["topic_name"]
bnew["id"] = b["id"]


#-b.iloc[:, 0:2].dot(M)

#bnew = (-b.iloc[:, 1:2]) @ (M) #TODO =========================================== #
#bnew = data.frame(as.matrix(-b[, 1:2]) % * % M)
#bnew = data.frame(as.matrix(-b[,2:1]) %*% M) # for 75, 70 # 계수 컬럼1과 컬럼2가 교환됨. for what?

anew = pd.DataFrame(np.array(-a.iloc[:, 0:2]) @ M, columns=["coordinate_1", "coordinate_2"]) #TODO ===========================================
#anew = (-a[:, 2:1]).dot(M)  # for 75






b = bnew
a = anew



In [180]:
################################################## TODO distance


a["dist"] = (a["coordinate_1"]**2 + a["coordinate_2"]**2)**0.5

a

#head(a)
#hist(a$dist)
#summary(a$dist)
#quantile(a$dist, c(0.8, 0.9))


a_new = a.loc[a["dist"]>1.5, :]


,coordinate_1,coordinate_2,dist


In [191]:
####################### TODO: 3dplot

#par(mfrow=c(1, 1))
#library(scatterplot3d)

new = pd.DataFrame({"x" : b["coordinate_1"],
                   "y" : b["coordinate_2"],
                   "z" : output_new["beta_estimate"],
                   "topics" : b["topic_name"]}) #word의 계수1, word의 계수2, beta의 측정치, word의 topic name,
                                                #output_new는 z_est, w_est, z.proc, w.proc으로 바꾼버전

word_cluster = KMeans(n_clusters=4).fit(output_new["z_estimate"])
word_cluster.labels_

topic_cluster = KMeans(n_clusters=4).fit(b.iloc[:, 0:2])
topic_cluster.labels_



#colors = topic_cluster["cluster"]
#topic_plot = scatterplot3d(new[, 1:3],
                           pch=16,
                           color=colors,
                           angle=50)  # TODO =================================
# word_plot$points3d(output_new$w_estimate,pch=8,color=color2)

array([3, 1, 1, 0, 0, 0, 1, 2, 1, 2, 1, 0, 2, 2, 2, 2, 3, 3, 3, 0, 3, 3,
       2, 3, 1, 2, 2, 2, 2, 0, 1, 3, 3, 1, 0, 1, 2, 1, 0, 2, 2, 3, 0, 3,
       0, 2, 0, 1, 0, 0, 2, 1, 0, 2, 2, 2, 3, 0, 0, 0, 3, 0, 0, 1, 1, 3,
       0, 3, 2, 1, 1, 0, 2, 2, 1, 0, 3, 2, 2, 2, 0, 0, 0, 0, 0, 3, 0, 3,
       0, 0, 0, 2, 3, 2, 2, 0, 1, 2, 3, 0, 0, 1, 1, 3, 1, 2, 2, 2, 3, 1,
       2, 2, 1, 0, 3, 0, 0, 0, 3, 2, 2, 1, 2, 3, 0, 1, 2, 2, 2, 3, 0, 3,
       3, 0, 2, 2, 2, 0, 2, 0, 3, 0, 0, 3, 2, 1, 2, 2, 3, 0, 3, 1, 0, 1,
       3, 2, 0, 3, 0, 2, 1, 2, 0, 3, 3, 0, 0, 2, 2, 0, 1, 3, 0, 2, 1, 1,
       1, 2, 2, 2, 0, 2, 2, 1, 3, 2, 3, 0, 1, 2, 0, 3, 2, 2, 0, 1, 3, 0,
       1, 2, 3, 3, 0, 2, 1, 1, 3, 2, 2, 0, 0, 2, 2, 1, 3, 3, 2, 2, 3, 3,
       0, 0, 1, 2, 0, 1, 2, 0, 2, 1, 0, 2, 3, 3, 2, 1, 2, 0, 2, 0, 3, 3,
       1, 1, 2, 1, 0, 0, 0, 2, 1, 1, 2, 2, 1, 2, 2, 2, 0, 2, 3, 0, 3, 0,
       1, 1, 0, 0, 3, 2, 3, 3, 3, 0, 1, 0, 0, 1, 3, 0, 2, 1, 3, 3, 1, 3,
       0, 2, 2, 1, 3, 3, 3, 0, 1, 0, 0, 0, 2, 2, 3,

In [192]:
wcss = [1, 2]
bet_tot = [1, 2]
bet = [1, 2]

ncluster = 5

data_set2 = data_m

from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from scipy.spatial.distance import euclidean


####################################################################################


def affinityMatrix(dif, K=20, sigma=0.5):
    diff = np.array(dif)
    
    N = diff.shape[0]
    diff = (diff + diff.transpose()) / 2
    np.fill_diagonal(diff, 0)
    sortedColumns = np.apply_along_axis(sorted, 0, diff).transpose()

    def finiteMean(x):
        return x[np.isfinite(x)].mean()
    
    eps = np.finfo("double").eps

    means = np.apply_along_axis(finiteMean, 1, sortedColumns[:, 0:K + 1]) + eps
    Sig = np.mean.outer(means, means) / 3 * 2 + diff / 3 + eps
    Sig[Sig <= eps] = eps

    densities = scipy.stats.norm.pdf(diff, 0, sigma * Sig)

    W = (densities + densities.transpose()) / 2
    
    return W


####################################################################################


for aa in range(2, (ncol(data_set2))-1):
    X = b.iloc[:, 0:2]
    #X = ex
    idist = np.zeros((X.shape[0], X.shape[0]))

    for i in range(X.shape[0]):
        for j in range(X.shape[0]):
            idist[i, j] = np.linalg.norm(X[i, :] - X[j, :]) 
            #euclidean(X[:, i], X[:, j])
            continue

    W = affinityMatrix(idist, K=aa)  # TODO =====================
    d = W.sum(axis=1)

    d[d == 0] = np.finfo("double").eps

    D = np.diag(d)

    L = D - W
    Di = np.diag(1 / sqrt(d))

    NL = Di @ L @ Di
    ev = np.linalg.eig(NL)[0]
    evec = np.linalg.eig(NL)[1]

    ix = pd.Index(abs(ev)).sort_values(return_indexer=True)[1]
    U = evec[:, ix[0:ncluster]]


    def normalize(x):
        return x / sqrt(sum(x ^ 2))


    U.apply(normalize, 1)

    final = KMeans(n_clusters=ncluster, random_state=0).fit(U)
    group = final.labels_

    totss = sum(sum(scale(X, axis=0, with_std=False) ** 2))
    tot_withinss = final.inertia_
    betweenss = totss - tot_withinss

    wcss[aa] = tot_withinss  # min #3:82%, 4: 61%, 5: 91%, 6:86%. 7:83%. 8: 85%, 9: 86%. 10: 87%
    bet_tot[aa] = betweenss / totss * 100  # max
    bet[aa] = betweenss

    # wcss[aa] < - final$tot.withinss  # min #3:82%, 4: 61%, 5: 91%, 6:86%. 7:83%. 8: 85%, 9: 86%. 10: 87%
    # bet_tot[aa] < - final$betweenss / final$totss * 100  # max
    # bet[aa] < - final$betweenss

# which.max(bet_tot)


,coordinate_1,coordinate_2,topic_name,id
0,-1.423824,0.673819,Outbreak,1
1,0.070232,0.325487,Emotional.Effect,2
2,1.066860,-0.236272,Symptom,3
3,0.492909,-1.231272,Hospital.Related,4
4,-1.419349,-0.141562,Immune.Reaction,5
5,0.258069,1.148658,Disease.Relations,6
6,-0.989446,1.572048,Policy,7
7,-1.417868,-0.055144,Psychological.Problem,8
8,-1.061147,-0.319857,Analysis.of.Transmission,9
9,0.494012,1.052818,Cardiovascular.System,10


In [ ]:

# TODO =====================
W = affinityMatrix(idist, K=which.max(bet_tot))  # TODO =====================

## cluster the topic using select number of cluster
# ncluster <- min(k)  # TODO =====================
ncluster = 3  # TODO =====================


group_index = c()

for i in range(1, max(group) + 1):
    group_index[i] = min(np.where(group == i))

# TODO =====================

In [233]:

word_position = pd.concat([data_word, a.iloc[:, 0:2]], axis=1)
word_position["dist"] = (word_position["coordinate_1"] ** 2 + word_position["coordinate_2"] ** 2) ** 0.5
word_new = word_position.loc[word_position["dist"] > 1.4, :]


#quantile(word_position$dist, c(0.05, 0.2))
word_new = word_position[word_position["dist"] < 0.41,]



In [ ]:

words = data_word
close_word_index = []

## cosine similarity

temp = pd.concat([b.iloc[:, 0:2], group], axis=1)


# head(temp)

## center of topic group


# temp2 = data.frame(temp % > % group_by(group) % > % summarise(x=mean(coordinate_1), y=mean(coordinate_2)))

temp2 = temp.groupby("group", axis=1).agg({"coordinate_1": "mean", 
                                           "coordinate_2": "mean"})
temp2.columns = ["x", "y"]

# head(temp2)

word_position = pd.concat([words, a[, 0:2]], axis=1)
close_word_index = []

for(i in 1:ncol(data_set2)){
  ## here 10
  #i = 1
  close_word_index[[i]] <- order(as.matrix(dist(rbind(b[i,1:2], a[,1:2])))[1,-1])[1:25]


In [191]:
##### Cluster_Group near words

#head(word_position)

for i in range(temp2.shape[0]):
    #i = 1
    ## here 10
    cluster_word_dist = []
    close_word_index = []
    
    np.delete(temp2, 0, 1)[i, :]
    
    for k in nrow(output_new["z_estimate"].shape[0]:
                  cluster_word_dist[k] = np.linalg.norm(np.delete(temp2, 0, 1)[i, :] - 
                                                        output_new["z_estimate"][k, :])
    
    close_word_index = order(as.matrix(cluster_word_dist))[1:25]
    

save.image("75_result.RData")



array([3, 1, 1, 0, 0, 0, 1, 2, 1, 2, 1, 0, 2, 2, 2, 2, 3, 3, 3, 0, 3, 3,
       2, 3, 1, 2, 2, 2, 2, 0, 1, 3, 3, 1, 0, 1, 2, 1, 0, 2, 2, 3, 0, 3,
       0, 2, 0, 1, 0, 0, 2, 1, 0, 2, 2, 2, 3, 0, 0, 0, 3, 0, 0, 1, 1, 3,
       0, 3, 2, 1, 1, 0, 2, 2, 1, 0, 3, 2, 2, 2, 0, 0, 0, 0, 0, 3, 0, 3,
       0, 0, 0, 2, 3, 2, 2, 0, 1, 2, 3, 0, 0, 1, 1, 3, 1, 2, 2, 2, 3, 1,
       2, 2, 1, 0, 3, 0, 0, 0, 3, 2, 2, 1, 2, 3, 0, 1, 2, 2, 2, 3, 0, 3,
       3, 0, 2, 2, 2, 0, 2, 0, 3, 0, 0, 3, 2, 1, 2, 2, 3, 0, 3, 1, 0, 1,
       3, 2, 0, 3, 0, 2, 1, 2, 0, 3, 3, 0, 0, 2, 2, 0, 1, 3, 0, 2, 1, 1,
       1, 2, 2, 2, 0, 2, 2, 1, 3, 2, 3, 0, 1, 2, 0, 3, 2, 2, 0, 1, 3, 0,
       1, 2, 3, 3, 0, 2, 1, 1, 3, 2, 2, 0, 0, 2, 2, 1, 3, 3, 2, 2, 3, 3,
       0, 0, 1, 2, 0, 1, 2, 0, 2, 1, 0, 2, 3, 3, 2, 1, 2, 0, 2, 0, 3, 3,
       1, 1, 2, 1, 0, 0, 0, 2, 1, 1, 2, 2, 1, 2, 2, 2, 0, 2, 3, 0, 3, 0,
       1, 1, 0, 0, 3, 2, 3, 3, 3, 0, 1, 0, 0, 1, 3, 0, 2, 1, 3, 3, 1, 3,
       0, 2, 2, 1, 3, 3, 3, 0, 1, 0, 0, 0, 2, 2, 3,

In [15]:

from pyarma import *



In [16]:
A = mat(4, 5, fill.randu)
B = mat(4, 5, fill.randu)
  
C = A*B.t()

C.print("C:")

C:
   1.2947   1.4712   1.6029   1.5053
   2.2507   1.6893   1.9306   2.2118
   1.8974   2.0922   2.3306   2.1280
   1.9538   1.5569   1.6389   2.1417
